## **Respuesta del Examen Parcial CC0C2**

### **Respuesta 1**

In [ ]:
import sys
from typing import Iterator, List, Optional, Tuple, Dict


def crear_data_loader(ruta: Optional[str], tamaño_batch: int) -> Iterator[List[str]]:
    """
    Crea un iterador que entrega el corpus en lotes de tamaño dado.
    Si ruta es None, genera un mini-corpus de ejemplo.
    """
    if ruta:
        with open(ruta, encoding='utf-8') as f:
            lines = [line.strip() for line in f if line.strip()]
    else:
        # Mini-corpus por defecto
        lines = [
            "The quick brown fox jumps over the lazy dog",
            "Byte-Pair Encoding is a simple data compression technique",
            "ChatGPT genera texto con un modelo de lenguaje"
        ]
    # Generar lotes
    def _batcher(data):
        for i in range(0, len(data), tamaño_batch):
            yield data[i:i + tamaño_batch]

    return _batcher(lines)


def entrenar_bpe_simple(
    corpus: List[str],
    num_merges: int
) -> List[Tuple[str, str]]:
    """
    Entrena reglas BPE de forma simple:
    - Tokeniza a nivel carácter con </w>
    - En cada paso, encuentra el par más frecuente y lo fusiona
    - Actualiza frecuencias sólo para palabras afectadas (actualización incremental)
    """
    # Tokenizar cada palabra del corpus en lista de símbolos
    words: List[List[str]] = []
    for sentence in corpus:
        for word in sentence.split():
            words.append(list(word) + ['</w>'])

    # Frecuencias iniciales de pares adyacentes
    def get_pair_freq(words_list: List[List[str]]) -> Dict[Tuple[str,str], int]:
        freqs: Dict[Tuple[str,str], int] = {}
        for symbols in words_list:
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i+1])
                freqs[pair] = freqs.get(pair, 0) + 1
        return freqs

    pair_freqs = get_pair_freq(words)
    rules: List[Tuple[str, str]] = []
    history: List[Tuple[int, Tuple[str, str], int, int]] = []

    for i in range(1, num_merges + 1):
        if not pair_freqs:
            break
        # Seleccionar el par más frecuente
        best_pair, freq_before = max(pair_freqs.items(), key=lambda x: x[1])
        a, b = best_pair
        merged = a + b
        # Aplicar fusión sólo en palabras que contienen el par
        for symbols in words:
            j = 0
            while j < len(symbols) - 1:
                if symbols[j] == a and symbols[j+1] == b:
                    symbols[j] = merged
                    symbols.pop(j+1)
                else:
                    j += 1
        # Recalcular frecuencias en todo el corpus (puede optimizarse)
        new_freqs = get_pair_freq(words)
        freq_after = new_freqs.get(best_pair, 0)
        history.append((i, best_pair, freq_before, freq_after))
        rules.append(best_pair)
        pair_freqs = new_freqs

    # Imprimir primeras 5 reglas con sus frecuencias
    print("Primeras 5 fusiones (iter, par, freq antes -> freq después):")
    for it, pair, bef, aft in history[:5]:
        print(f"{it}: {pair} -> {bef} -> {aft}")
    return rules


def tokenizar_con_bpe(texto: str, reglas: List[Tuple[str,str]]) -> List[str]:
    """
    Tokeniza texto aplicando reglas BPE en orden.
    """
    tokens: List[str] = []
    for word in texto.split():
        symbols = list(word) + ['</w>']
        for a, b in reglas:
            merged = a + b
            j = 0
            while j < len(symbols) - 1:
                if symbols[j] == a and symbols[j+1] == b:
                    symbols[j] = merged
                    symbols.pop(j+1)
                else:
                    j += 1
        # Cada símbolo (incluyendo los merges) conserva </w> en sus tokens
        for sym in symbols:
            tokens.append(sym)
    return tokens


def detokenizar_bpe(tokens: List[str]) -> str:
    """
    Reconstruye texto original a partir de tokens BPE.
    Maneja tokens que contienen el sufijo '</w>'.
    """
    result = []
    for tok in tokens:
        if tok.endswith('</w>'):
            # Sufijo indica fin de palabra
            word = tok[:-4]
            result.append(word)
            result.append(' ')
        else:
            result.append(tok)
    text = ''.join(result).strip()
    return text


if __name__ == "__main__":
    # Demo mínima
    corpus = [
        "low low lower",
        "lowest lower lowest"
    ]
    num_merges = 20
    reglas = entrenar_bpe_simple(corpus, num_merges)

    # Vocabulario inicial y final
    vocab_inicial = set()
    for sent in corpus:
        for w in sent.split():
            vocab_inicial.update(list(w) + ['</w>'])
    vocab_final = set(tokenizar_con_bpe(' '.join(corpus), reglas))

    # Round-trip con asserts
    frases = ["low lower", "lowest low"]
    for original in frases:
        tokens = tokenizar_con_bpe(original, reglas)
        recovered = detokenizar_bpe(tokens)
        assert recovered == original, f"Round-trip falló: {original}"
        print("Round-trip OK para:", original)

    # Comprobaciones adicionales
    assert len(vocab_inicial) > len(vocab_final), "El vocabulario no se ha reducido"
    assert any(pair in reglas for pair in [("l","o"), ("e","s")]), "No se detectaron pares comunes"


### **Respuesta 2**

In [ ]:
import numpy as np
from typing import Dict, List, Tuple


def crear_embeddings_semilla(vocab: List[str], dim: int = 50, seed: int = 42) -> Dict[str, np.ndarray]:
    """
    Genera embeddings reproducibles para cada palabra del vocabulario,
    usando una distribución gaussiana estándar.
    """
    rng = np.random.default_rng(seed)
    embeddings = {word: rng.standard_normal(dim) for word in vocab}
    return embeddings


def similitud_coseno(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    Calcula la similitud coseno entre dos vectores.
    """
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot / (norm1 * norm2)


def encontrar_palabras_mas_similares(
    palabra_objetivo: str,
    embeddings: Dict[str, np.ndarray],
    top_n: int = 5
) -> List[Tuple[str, float]]:
    """
    Devuelve las top_n palabras más similares a la palabra_objetivo
    según similitud coseno. Maneja out-of-vocabulary.
    """
    if palabra_objetivo not in embeddings:
        raise ValueError(f"'{palabra_objetivo}' no está en el vocabulario.")
    vec_target = embeddings[palabra_objetivo]
    scores = []
    for w, vec in embeddings.items():
        if w == palabra_objetivo:
            continue
        score = similitud_coseno(vec_target, vec)
        scores.append((w, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_n]


def resolver_analogia(
    w1: str, w2: str, w3: str,
    embeddings: Dict[str, np.ndarray]
) -> str:
    """
    Resuelve analogía w1:w2 :: w3:? usando vec(w2)-vec(w1)+vec(w3)
    y devuelve la palabra más similar, excluyendo w1,w2,w3.
    """
    for w in (w1, w2, w3):
        if w not in embeddings:
            raise ValueError(f"'{w}' no está en el vocabulario.")
    vec = embeddings[w2] - embeddings[w1] + embeddings[w3]
    best_word = None
    best_score = -np.inf
    for w, emb in embeddings.items():
        if w in (w1, w2, w3):
            continue
        score = similitud_coseno(vec, emb)
        if score > best_score:
            best_score = score
            best_word = w
    return best_word


if __name__ == "__main__":
    # Definir un vocabulario pequeño con pares de género y profesiones
    vocab = [
        'hombre', 'mujer', 'rey', 'reina', 'doctor', 'doctora',
        'maestro', 'maestra', 'ingeniero', 'ingeniera', 'hospital',
        'escuela', 'ciudad', 'palacio', 'profesor', 'profesora',
        'rey', 'reina', 'príncipe', 'princesa', 'actor', 'actriz',
        'rey', 'reina', 'agua', 'fuego', 'tierra', 'aire', 'sol', 'luna'
    ]

    # 1) Crear embeddings sintéticos
    embeddings = crear_embeddings_semilla(vocab, dim=50, seed=123)

    # 2) Búsqueda de vecinos más similares para 3 palabras
    prueba_palabras = ['doctor', 'mañana', 'ingeniero']
    for palabra in ['doctor', 'maestro', 'ingeniero']:
        vecinos = encontrar_palabras_mas_similares(palabra, embeddings, top_n=5)
        print(f"Vecinos de '{palabra}':")
        for w, score in vecinos:
            print(f"  {w}: {score:.4f}")
        print()

    # 3) Resolver analogías
    analogias = [
        ('rey', 'hombre', 'mujer'),
        ('doctor', 'hospital', 'maestra')
    ]
    for w1, w2, w3 in analogias:
        respuesta = resolver_analogia(w1, w2, w3, embeddings)
        print(f"{w1} : {w2} :: {w3} : {respuesta}")
    print()

    # 4) Experimento de sesgo
    # Medir similitud de 'ingeniero' con 'hombre' y 'mujer'
    sim_h = similitud_coseno(embeddings['ingeniero'], embeddings['hombre'])
    sim_m = similitud_coseno(embeddings['ingeniero'], embeddings['mujer'])
    print("Similitud(ingeniero, hombre):", f"{sim_h:.4f}")
    print("Similitud(ingeniero, mujer):", f"{sim_m:.4f}")
    if sim_h > sim_m:
        print("Potencial sesgo: 'ingeniero' está más cerca de 'hombre'.")
    else:
        print("Potencial sesgo: 'ingeniero' está más cerca de 'mujer'.")
    print()

    print("Nota: con vectores aleatorios, estas correlaciones son fortuitas.")



### **Respuesta 3**

#### **Diferencias entre greedy decoding, beam Search y muestreo con temperatura**

Estos son tres métodos para seleccionar el siguiente token en una secuencia durante la fase de generación (inferencia) de un modelo de lenguaje.

* **Greedy decoding:**
    * **Explicación:** En cada paso de tiempo, selecciona el token con la probabilidad más alta según la salida del modelo. Es el enfoque más simple y rápido.
    * **Ventajas:** Muy rápido y computacionalmente barato.
    * **Desventajas:** Puede llevar a secuencias subóptimas. Una elección localmente óptima (la palabra más probable ahora) puede cerrar la puerta a una secuencia global mucho mejor. A menudo genera texto repetitivo y predecible.

* **Beam search:**
    * **Explicación:** En lugar de mantener una única secuencia candidata (la mejor), mantiene `k` secuencias candidatas (el "haz" o *beam*). En cada paso, expande cada una de las `k` secuencias con todos los posibles siguientes tokens del vocabulario. De todas las nuevas secuencias generadas, se queda con las `k` mejores según su probabilidad acumulada (generalmente se usa la suma de log-probabilidades para evitar underflow numérico).
    * **Ventajas:** Explora un espacio de búsqueda más amplio que el greedy decoding, lo que generalmente conduce a secuencias de mayor calidad y más probables.
    * **Desventajas:** Es computacionalmente más costoso. Requiere `k` veces más memoria y cómputo que el greedy search. Puede seguir favoreciendo secuencias de alta probabilidad pero "aburridas" o repetitivas.

* **Muestreo con temperatura:**
    * **Explicación:** Introduce aleatoriedad en la selección de tokens. Antes de aplicar la función softmax a las salidas del modelo (logits), estos se dividen por un valor de **temperatura** ($T$).
        * Si $T \to 0$, el muestreo se vuelve determinista, similar al greedy decoding.
        * Si $T = 1$, se muestrea de la distribución de probabilidad original del modelo.
        * Si $T > 1$, la distribución de probabilidad se vuelve más uniforme, aumentando la probabilidad de que se seleccionen tokens menos probables. Esto introduce más diversidad y "creatividad" en el texto.
    * **Ventajas:** Permite controlar el equilibrio entre aleatoriedad y determinismo. Útil para tareas creativas donde la diversidad es deseable.
    * **Desventajas:** Una temperatura demasiado alta puede generar texto incoherente y sin sentido. El resultado no es reproducible a menos que se fije la semilla aleatoria.

#### **Uso de *teacher forcing**

El **teacher forcing** es una técnica de entrenamiento para modelos recurrentes (RNN, LSTM, etc.) en tareas de secuencia a secuencia.

* **¿Cuándo y por qué emplearlo?:** Se emplea durante el **entrenamiento**. En lugar de alimentar al siguiente paso de tiempo la salida que el propio modelo predijo en el paso anterior, se le alimenta el **token correcto** (la verdad fundamental o *ground truth*) de la secuencia objetivo.
* **Ventajas:**
    1.  **Acelera la convergencia:** Al recibir siempre la entrada correcta, el modelo aprende más rápido y el entrenamiento es mucho más estable.
    2.  **Paralelización:** Permite que los cálculos para cada paso de tiempo se realicen en paralelo, ya que la entrada en el tiempo $t$ no depende de la salida del modelo en $t-1$.
* **Problema asociado (exposure bias):** Durante la inferencia, el modelo solo tiene acceso a sus propias predicciones, que pueden contener errores. Como nunca fue expuesto a sus propios errores durante el entrenamiento, puede cometer un error y desviarse significativamente, generando secuencias de baja calidad.


#### **Limitación del vector de contexto y la atención de Bahdanau**

* **Problema del vector de contexto único:** En la arquitectura Seq2Seq clásica, el *encoder* comprime toda la información de la oración de entrada en un único vector de tamaño fijo, llamado **vector de contexto**. Este vector es la única información que el *decoder* recibe sobre la entrada. Para oraciones largas, es extremadamente difícil (si no imposible) comprimir todos los matices y dependencias en este vector sin una pérdida significativa de información. Esto actúa como un **cuello de botella** (*bottleneck*), degradando el rendimiento en secuencias largas.

* **Atención "global" de Bahdanau:** La atención de Bahdanau soluciona este problema permitiendo que el *decoder* "mire" a todas las salidas ocultas del *encoder* en cada paso de decodificación.
    1.  En cada paso del decoder (al generar una palabra), el estado oculto actual del decoder se compara con **todos** los estados ocultos del encoder.
    2.  Esta comparación genera una serie de **pesos de atención** (o *alignment scores*), que se normalizan con una función softmax. Estos pesos indican qué partes de la oración de entrada son más relevantes para generar la palabra actual.
    3.  Se calcula un **vector de contexto dinámico** como una suma ponderada de los estados ocultos del encoder, usando los pesos de atención.
    4.  Este vector de contexto dinámico, específico para el paso de tiempo actual, se concatena con el estado oculto del decoder y se utiliza para predecir la siguiente palabra.

De esta forma, el modelo no depende de un único vector fijo, sino que crea un atajo focalizado a las partes relevantes de la entrada en cada paso, solucionando el problema del cuello de botella.


#### **Función de la máscara de atención con teacher forcing**

En arquitecturas como los Transformers, que no son inherentemente secuenciales, se puede procesar toda la secuencia a la vez. Cuando se usa *teacher forcing* en el *decoder* de un Transformer, se le alimenta la secuencia de salida completa.

La **máscara de atención** (o *look-ahead mask*) es crucial aquí. Su función es asegurar que, al predecir el token en la posición $t$, el modelo solo pueda atender a los tokens en posiciones anteriores ($< t$) y no a los futuros ($\ge t$). Impide que el modelo "haga trampa" mirando la respuesta correcta que viene después en la secuencia. Esto preserva la propiedad **autorregresiva** del modelo, forzándolo a aprender a predecir el siguiente token basándose únicamente en los tokens anteriores.


#### **Gradient Clipping**

El *gradient clipping* es una técnica para mitigar el problema de la **explosión de gradientes**, común en las RNNs. La explosión de gradientes ocurre cuando los gradientes crecen exponencialmente durante la retropropagación a través del tiempo, resultando en actualizaciones de pesos enormes que desestabilizan el entrenamiento (a menudo resultando en `NaN`).

* **Gradient clipping por valor (clip by value):** Establece un umbral mínimo y máximo (ej., `[-c, c]`). Si un gradiente está fuera de este rango, se recorta a ese valor límite. `grad = max(min_val, min(max_val, grad))`.
* **Gradient clipping por norma (clip by norm):** Calcula la norma L2 de todo el vector de gradientes de un parámetro (o de todos los parámetros juntos). Si esta norma excede un umbral `max_norm`, todo el vector de gradientes se reescala para que su norma sea igual a `max_norm`, manteniendo su dirección original. $$\text{Si } \|\mathbf{g}\| > \text{max\_norm}, \text{ entonces } \mathbf{g} \leftarrow \frac{\text{max\_norm}}{\|\mathbf{g}\|} \mathbf{g}$$

**Efecto negativo de un umbral demasiado bajo:** Si el umbral de *clipping* es demasiado bajo, el modelo puede aprender muy lentamente. Se estarían "frenando" incluso las actualizaciones de gradientes legítimamente grandes que son necesarias para un aprendizaje rápido y efectivo, lo que podría impedir que el modelo converja a una buena solución.

#### **Teacher-forcing ratio y estrategias de reducción**

La ***teacher-forcing ratio*** es la probabilidad con la que se utilizará la técnica de *teacher forcing* en un paso de entrenamiento. Un ratio de 1.0 significa usar siempre *teacher forcing*, y un ratio de 0.0 significa no usarlo nunca (el modelo se alimenta de sus propias predicciones, como en la inferencia).

Reducir este ratio a lo largo del entrenamiento ayuda a mitigar el *exposure bias*. Al exponer gradualmente al modelo a sus propias predicciones (y errores potenciales), se vuelve más robusto y su rendimiento en la inferencia mejora.

**Dos escenarios para reducirla:**

1.  **Decaimiento lineal:**
    * **Propuesta:** Empezar con un ratio de 1.0 y disminuirlo linealmente en cada época (o cada N iteraciones) hasta que llegue a 0.0 (o a un valor pequeño como 0.1) hacia el final del entrenamiento.
    * **Justificación:** Es una estrategia simple y efectiva. Al principio, cuando el modelo es inestable, se beneficia de la guía constante del *teacher forcing*. A medida que aprende, se le "quita" gradualmente, forzándolo a aprender a corregir sus propios errores y a ser más robusto ante las condiciones de la inferencia.

2.  **Muestreo programado (scheduled sampling):**
    * **Propuesta:** En lugar de decaer el ratio para todo el batch, se puede decidir estocásticamente para cada instancia o paso de tiempo si usar *teacher forcing* o no, basándose en un ratio que decae. Una versión más avanzada es hacer que la probabilidad de usar la predicción del modelo en lugar del *ground truth* sea una función del número de época (ej., `k / (k + exp(epoch / k))` para alguna constante `k`).
    * **Justificación:** Introduce una transición más suave y estocástica entre el entrenamiento y la inferencia. Al forzar al modelo a enfrentarse a sus errores de forma aleatoria, se puede lograr una mayor generalización y robustez en comparación con un decaimiento determinista.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
from collections import Counter
import time
from sklearn.metrics import precision_score, recall_score, f1_score

# 0. Datos 
sentences = [
    "I love this place!", "Worst service ever.", "The food was amazing.",
    "I will never come back.", "Absolutely fantastic experience!", "Terrible, simply terrible.",
    "Not bad at all.", "It was okay, nothing special.", "I’m delighted with the result.",
    "This is disappointing.", "Great job, team!", "I hate waiting so long.",
    "Superb quality and taste.", "The product broke instantly.", "Highly recommend it.",
    "Save your money, skip this.", "Totally worth the price.", "Service was rude and slow.",
    "Exceeded my expectations.", "I regret buying this."
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

# 1. Preprocesamiento

# a. Tokenización (simple split)
tokenized_sentences = [s.lower().split() for s in sentences]

# b. Creación del vocabulario ordenado por frecuencia, añadiendo tokens especiales
word_counts = Counter(word for s in tokenized_sentences for word in s)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
word_to_idx = {word: i+2 for i, word in enumerate(vocab)}
word_to_idx['<PAD>'] = 0
word_to_idx['<UNK>'] = 1
idx_to_word = {i: w for w, i in word_to_idx.items()}
vocab_size = len(word_to_idx)

# c. Conversión a índices y d. Padding
indexed = [
    [word_to_idx.get(w, word_to_idx['<UNK>']) for w in s]
    for s in tokenized_sentences
]
max_len = max(len(seq) for seq in indexed)
padded = np.array([
    seq + [word_to_idx['<PAD>']] * (max_len - len(seq))
    for seq in indexed
])

print(f"Tamaño del vocabulario: {vocab_size}")
print(f"Longitud máxima de secuencia: {max_len}\n")

# Datos PyTorch
features = torch.from_numpy(padded).long()
targets  = torch.tensor(labels).float().unsqueeze(1)

dataset = TensorDataset(features, targets)
train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=4)

# 2. Construcción del modelo

class SentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                 rnn_type='RNN', n_layers=1, dropout=0.2):
        super(SentimentClassifier, self).__init__()
        # 2.1 Embedding con padding_idx=0
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # 2.2 Selección de RNN/GRU/LSTM y ajuste de dropout solo si n_layers>1
        dp = dropout if n_layers > 1 else 0.0
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                               dropout=dp, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, n_layers,
                              dropout=dp, batch_first=True)
        else:
            self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers,
                              dropout=dp, batch_first=True)

        # 2.3 Dropout y capa final
        self.dropout = nn.Dropout(dropout)
        self.fc      = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # x: [batch, seq_len]
        embedded = self.embedding(x)           # [batch, seq_len, emb_dim]
        rnn_out, hidden = self.rnn(embedded)
        # hidden:
        #  - RNN/GRU -> tensor [n_layers, batch, hidden_dim]
        #  - LSTM    -> tuple (h_n, c_n), cada uno [n_layers, batch, hidden_dim]
        if isinstance(hidden, tuple):
            # LSTM
            h_n, c_n    = hidden
            last_hidden = h_n[-1]             # [batch, hidden_dim]
        else:
            # RNN o GRU
            last_hidden = hidden[-1]          # [batch, hidden_dim]

        out = self.dropout(last_hidden)       # [batch, hidden_dim]
        out = self.fc(out)                    # [batch, 1]
        return out

#3. Entrenamiento y 4. Evaluación 

def train_and_evaluate(modelo, model_name):
    print(f"-{model_name}")
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(modelo.parameters(), lr=0.005)
    epochs    = 25

    start = time.time()
    modelo.train()
    for _ in range(epochs):
        for xb, yb in train_loader:
            optimizer.zero_grad()
            logits = modelo(xb)
            loss   = criterion(logits, yb)
            loss.backward()
            optimizer.step()
    end = time.time()
    print(f"Tiempo de entrenamiento: {end-start:.3f}s")

    # Evaluación
    modelo.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            out = modelo(xb)
            preds = torch.round(torch.sigmoid(out))
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(yb.cpu().numpy())

    all_preds  = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()
    p = precision_score(all_labels, all_preds, zero_division=0)
    r = recall_score(all_labels, all_preds, zero_division=0)
    f = f1_score(all_labels, all_preds, zero_division=0)
    print(f"Precisión: {p:.4f}, Recall: {r:.4f}, F1: {f:.4f}\n")

# 5. Comparación de modelos 

embedding_dim = 32
hidden_dim    = 32
n_layers      = 1

# 5.1 RNN simple
model_rnn = SentimentClassifier(vocab_size, embedding_dim, hidden_dim,
                                rnn_type='RNN', n_layers=n_layers, dropout=0.2)
train_and_evaluate(model_rnn, "RNN Simple")

# 5.2 LSTM
model_lstm = SentimentClassifier(vocab_size, embedding_dim, hidden_dim,
                                 rnn_type='LSTM', n_layers=n_layers, dropout=0.2)
train_and_evaluate(model_lstm, "LSTM")


**Simulador de decoder Seq2Seq con beam search**

In [ ]:
import numpy as np

# 1. Simulación del entorno ---

# Vocabulario pequeño
vocab = {
    '<SOS>': 0, 'A': 1, 'B': 2, 'C': 3, '<EOS>': 4, 'D': 5
}
idx_to_word = {i: w for w, i in vocab.items()}

def funcion_paso_modelo(secuencia_parcial_indices):
    """
    Función simulada que devuelve una distribución de probabilidad
    sobre el siguiente token. Tiene una lógica simple y predecible.
    """
    last_token_idx = secuencia_parcial_indices[-1]
    
    # Probabilidades por defecto (distribución uniforme)
    probs = np.full(len(vocab), 0.1)

    if last_token_idx == vocab['<SOS>']:
        # Al inicio, favorecer 'A'
        probs[vocab['A']] = 0.8
    elif last_token_idx == vocab['A']:
        # Después de 'A', favorecer 'B' o 'C'
        probs[vocab['B']] = 0.6
        probs[vocab['C']] = 0.3
    elif last_token_idx == vocab['B']:
        # Después de 'B', favorecer 'C' o '<EOS>'
        probs[vocab['C']] = 0.5
        probs[vocab['<EOS>']] = 0.4
    elif last_token_idx == vocab['C']:
        # Después de 'C', casi siempre terminar
        probs[vocab['<EOS>']] = 0.9
    
    # Normalizar para que sumen 1
    probs /= np.sum(probs)
    
    return {idx: p for idx, p in enumerate(probs)}

# 2. Implementación de beam search

def beam_search_decoder(k_beam, max_longitud_secuencia, funcion_paso_modelo):
    """
    Implementa la decodificación con beam search.
    """
    # Inicializar con el token <SOS>
    # Cada 'haz' es una tupla: (secuencia, log_prob_acumulada)
    k_beams = [([vocab['<SOS>']], 0.0)]
    
    # Lista para guardar las secuencias completas que terminan en <EOS>
    completed_sequences = []

    for _ in range(max_longitud_secuencia):
        all_candidates = []
        
        # 1. Expandir cada beam
        for seq, score in k_beams:
            # Si el último token es <EOS>, la secuencia está completa
            if seq[-1] == vocab['<EOS>']:
                completed_sequences.append((seq, score))
                continue

            # Obtener las probabilidades del siguiente token del 'modelo'
            next_token_probs = funcion_paso_modelo(seq)
            
            # Crear nuevos candidatos a partir del beam actual
            for token_idx, prob in next_token_probs.items():
                if prob == 0: continue # Evitar log(0)
                
                new_seq = seq + [token_idx]
                # Usamos log-probabilidades para estabilidad numérica y porque sumar es más rápido
                new_score = score + np.log(prob)
                all_candidates.append((new_seq, new_score))

        # Si no hay candidatos para expandir y ya hay secuencias completas, terminamos
        if not all_candidates:
            break
            
        # 2. Ordenar todos los candidatos y podar (pruning)
        # Se ordena por puntuación (mayor es mejor)
        ordered_candidates = sorted(all_candidates, key=lambda x: x[1], reverse=True)
        
        # 3. Seleccionar los k_beam mejores para el siguiente paso
        # Se toman 'k_beam' menos el número de secuencias que ya hemos completado
        k_beams = ordered_candidates[:k_beam - len(completed_sequences)]

        # Si todos los beams activos han generado secuencias completas, podemos parar antes
        if all(s[-1] == vocab['<EOS>'] for s, _ in k_beams) and len(k_beams) > 0:
            completed_sequences.extend(k_beams)
            break

    # Si el bucle termina por max_longitud, agregar los haces actuales a las completadas
    completed_sequences.extend(k_beams)
    
    # Ordenar las secuencias completadas finales por su puntuación
    # Se puede normalizar por longitud para no penalizar secuencias largas: score / len(seq)
    final_sorted_sequences = sorted(completed_sequences, key=lambda x: x[1] / len(x[0]), reverse=True)
    
    return final_sorted_sequences

# 3. Simulación y comparación

def run_simulation(k):
    print(f"Ejecutando beam search con k_beam = {k}")
    resultados = beam_search_decoder(
        k_beam=k,
        max_longitud_secuencia=5,
        funcion_paso_modelo=funcion_paso_modelo
    )
    
    print("Secuencias generadas (ordenadas por puntuación/longitud):")
    for seq_indices, score in resultados:
        seq_words = [idx_to_word[i] for i in seq_indices]
        print(f"  - Secuencia: {' '.join(seq_words)}")
        print(f"    Log-Prob score: {score:.4f}")
        print(f"    Normalized score: {score/len(seq_indices):.4f}\n")

run_simulation(k=2)
run_simulation(k=3)

### **Respuesta 4**

#### **Cálculo de pesos de atención aditiva (Bahdanau)**

In [ ]:
import numpy as np

def softmax(x):
    """Calcula la función softmax de manera numéricamente estable."""
    e_x = np.exp(x - np.max(x)) # Restar el máximo mejora la estabilidad
    return e_x / e_x.sum(axis=0)

def calcular_pesos_atencion_bahdanau(estado_decoder_anterior, estados_encoder, Wa, Ua, va):
    """
    Calcula los pesos de atención de Bahdanau.
    
    Args:
        estado_decoder_anterior (np.array): Vector del estado oculto anterior del decoder (s_{t-1}).
        estados_encoder (np.array): Matriz con los estados ocultos del encoder (h_j).
        Wa (np.array): Matriz de pesos para el estado del decoder.
        Ua (np.array): Matriz de pesos para los estados del encoder.
        va (np.array): Vector de pesos para calcular el score.
        
    Returns:
        np.array: Vector de pesos de atención (alpha_t).
    """
    scores = []
    
    # Proyectamos el estado del decoder una sola vez
    s_proyectado = np.dot(Wa, estado_decoder_anterior) # W_a * s_{t-1}
    
    # Iteramos sobre cada estado oculto del encoder
    for h_j in estados_encoder:
        # Proyectamos el estado del encoder
        h_proyectado = np.dot(Ua, h_j) # U_a * h_j
        
        # Calculamos la energía (score)
        # v_a^T * tanh(W_a*s_{t-1} + U_a*h_j)
        suma_proyecciones = s_proyectado + h_proyectado
        energia = np.dot(va.T, np.tanh(suma_proyecciones))
        scores.append(energia[0]) # El resultado es una matriz 1x1, tomamos el escalar
        
    # Aplicamos softmax a todos los scores para obtener los pesos de atención
    pesos_atencion = softmax(np.array(scores))
    
    return np.array(scores), pesos_atencion

# Simulación
np.random.seed(42)

# Dimensiones
dim_estado = 4
dim_atencion = 3
num_estados_encoder = 3

# 1. Definir datos de ejemplo
estado_decoder_anterior = np.random.rand(dim_estado)
# Creamos una matriz donde cada fila es un estado del encoder
estados_encoder = np.random.rand(num_estados_encoder, dim_estado) 

# 2. Inicializar matrices de pesos con dimensiones compatibles
Wa = np.random.rand(dim_atencion, dim_estado) # (3, 4)
Ua = np.random.rand(dim_atencion, dim_estado) # (3, 4)
va = np.random.rand(dim_atencion, 1)          # (3, 1)

# 3. Llamar a la función y mostrar resultados
print("Datos de entrada y pesos ")
print(f"Dimensiones de Wa: {Wa.shape}")
print(f"Dimensiones de Ua: {Ua.shape}")
print(f"Dimensiones de va: {va.shape}\n")
print(f"Estado del decoder s_{{t-1}} (dim {estado_decoder_anterior.shape}):\n{estado_decoder_anterior}\n")
print(f"Estados del encoder H (dim {estados_encoder.shape}):\n{estados_encoder}\n")

scores_calculados, pesos_finales = calcular_pesos_atencion_bahdanau(
    estado_decoder_anterior, estados_encoder, Wa, Ua, va
)

print("Resultados del cálculo de atención")
print(f"Scores de energía (sin normalizar):\n{scores_calculados}\n")
print(f"Pesos de atención finales (después de softmax):\n{pesos_finales}\n")
print(f"Verificación: La suma de los pesos es: {np.sum(pesos_finales):.4f}")


#### Explicación

El objetivo es calcular un peso $\alpha_{tj}$ para cada estado del encoder $h_j$ que nos diga cuán "relevante" es ese estado para generar la salida actual, dado el estado del decoder $s_{t-1}$.

La fórmula es: $$\alpha_{tj} = \frac{\exp(e_{tj})}{\sum_{k=1}^{N} \exp(e_{tk})}$$Donde el score de energía $e_{tj}$ se calcula como:$$e_{tj} = v_a^T \tanh(W_a s_{t-1} + U_a h_j)$$

Analicemos las dimensiones en la implementación:

1.  **$W_a s_{t-1}$ y $U_a h_j$**:
    * $s_{t-1}$ es un vector de dimensión `(4,)`.
    * $h_j$ es un vector de dimensión `(4,)`.
    * Para poder sumarlos, ambos deben ser proyectados al mismo espacio, el "espacio de atención", que definimos con `dim_atencion = 3`.
    * **$W_a$** y **$U_a$** son las matrices de proyección. Para transformar un vector de `(4,)` a `(3,)`, sus dimensiones deben ser `(3, 4)`.
    * El cálculo `np.dot(Wa, s_t-1)`: `(3, 4) @ (4,)` resulta en un vector de `(3,)`.
    * El cálculo `np.dot(Ua, h_j)`: `(3, 4) @ (4,)` también resulta en un vector de `(3,)`.

2.  **$\tanh(\dots)$**:
    * La suma de las dos proyecciones da un vector de `(3,)`.
    * La función `tanh` se aplica elemento a elemento, por lo que el resultado sigue siendo un vector de `(3,)`.

3.  **$v_a^T \tanh(\dots)$**:
    * El propósito de **$v_a$** es tomar este vector de `dim_atencion` y colapsarlo en un único número escalar: el **score de energía**.
    * Definimos $v_a$ con dimensión `(3, 1)`. Su transpuesta $v_a^T$ tiene dimensión `(1, 3)`.
    * El cálculo `np.dot(va.T, tanh_output)`: `(1, 3) @ (3,)` resulta en un escalar (o una matriz `(1,1)`), que es el score $e_{tj}$.

4.  **Softmax**:
    * Repetimos este proceso para cada uno de los `num_estados_encoder = 3` estados de $h$.
    * Obtenemos 3 scores, que agrupamos en un vector `[e_t1, e_t2, e_t3]`.
    * La función `softmax` se aplica a este vector para convertir los scores en una distribución de probabilidad, cuyos valores (los pesos de atención $\alpha_t$) suman 1.


#### **Análisis de autoatención**

Aquí exploramos los componentes fundamentales de la autoatención: las matrices **Query (Q)**, **Key (K)** y **Value (V)**, y cómo interactúan para calcular los scores de atención.

In [ ]:
import numpy as np

# Para reproducibilidad
np.random.seed(99)

# Contexto y datos iniciales
# Secuencia de 3 palabras, cada una con un embedding de dimensión 4
seq_len = 3
embedding_dim = 4
dk = 3 # Dimensión para Q, K, V

# Matriz de embeddings de entrada X (cada fila es una palabra)
embeddings_entrada = np.random.rand(seq_len, embedding_dim)

# Matrices de pesos para proyectar los embeddings
Wq = np.random.rand(embedding_dim, dk) # (4, 3)
Wk = np.random.rand(embedding_dim, dk) # (4, 3)
Wv = np.random.rand(embedding_dim, dk) # (4, 3)

# Implementación

# 1. Calcular matrices Q, K, V
# X @ Wq -> (3, 4) @ (4, 3) = (3, 3)
Q = np.dot(embeddings_entrada, Wq)
K = np.dot(embeddings_entrada, Wk)
V = np.dot(embeddings_entrada, Wv)

# 2. Calcular los scores de atención (sin escalar ni softmax)
# Q @ K.T -> (3, 3) @ (3, 3) = (3, 3)
scores = np.dot(Q, K.T)

# Salidas
print("Datos de entrada y pesos")
print(f"Embeddings de entrada X (dim {embeddings_entrada.shape}):\n{embeddings_entrada}\n")
print(f"Pesos W_Q (dim {Wq.shape}):\n{Wq}\n")

print("Matrices Q, K, V")
print(f"Matriz Q (dim {Q.shape}):\n{Q}\n")
print(f"Matriz K (dim {K.shape}):\n{K}\n")
print(f"Matriz V (dim {V.shape}):\n{V}\n")

print("Matriz de scores de atención")
print(f"Scores = QK^T (dim {scores.shape}):\n{scores}\n")

#### **Análisis conceptual**

1.  **¿Qué representan las filas y columnas de la matriz $QK^T$ resultante?**
    * Tanto las **filas** como las **columnas** representan las palabras (o tokens) de la secuencia de entrada en su respectivo orden.
    * La matriz $QK^T$ es la **matriz de scores de energía**. El valor en la posición $(i, j)$ representa el **score de similitud o alineación** entre la **Query** de la palabra $i$ y la **Key** de la palabra $j$. Es una medida de cuán "relevante" es la palabra $j$ para la palabra $i$ en el contexto de esta secuencia.

2.  **Si un valor en la posición $(i, j)$ de $QK^T$ es alto, ¿qué implicaría?**
    * Un valor alto en $(i, j)$ implica una **fuerte afinidad** entre la consulta de la palabra $i$ y la clave de la palabra $j$. Antes de la normalización con softmax, esto significa que el modelo ha aprendido (a través de las proyecciones $W_Q$ y $W_K$) que para construir la nueva representación contextual de la palabra $i$, debe **prestar mucha atención** a la información contenida en la palabra $j$. Después del softmax, este score alto se convertirá en un peso de atención cercano a 1, mientras que otros scores bajos se volverán cercanos a 0.

3.  **¿Cómo se usaría luego la matriz V?**
    * El paso final consiste en crear las nuevas representaciones de las palabras. Esto se hace calculando una **suma ponderada de todos los vectores value (V)**, donde los pesos son los scores de atención que acabamos de discutir (después de aplicar el escalado y el softmax).
    * La fórmula es: $\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$.
    * En la práctica, la matriz de pesos de atención (el resultado del softmax) se multiplica por la matriz V. La **fila $i$** de la matriz resultante es la nueva representación de la palabra $i$. Esta nueva representación ya no es el embedding original, sino una **versión contextualizada**: una mezcla de los "valores" de todas las palabras de la secuencia, ponderada por la relevancia que cada palabra tiene para la palabra $i$.